In [60]:
""" Classes and examples for searching for flights using SkyPicker. """
from __future__ import unicode_literals, absolute_import, generators, \
    print_function

import requests
from datetime import datetime, timedelta
import numpy as np
import json
import pandas as pd

### for the geodesic module later on down
import os
import math
import logging
from pyproj import Geod
from shapely.geometry import LineString, mapping
from fiona import collection
from fiona.transform import transform_geom
from fiona.crs import from_epsg



In [61]:
class SkyPickerApi(object):
    """ SkyPicker API. """
    def __init__(self):
        """ Initializes the API object with URL attributes. """
        self.base_url = 'https://api.skypicker.com/'
        self.path = ''
        self.param_str = ''

    @property
    def full_url(self):
        """ Returns the full URL for requesting the data. """
        return '{}{}{}'.format(self.base_url, self.path, self.param_str)

    def get_request(self):
        """ Requests the API endpoint and returns the response """
        headers = {'content-type': 'application/json'}
        resp = requests.get(self.full_url, headers=headers)
        return resp.json()
    

    def search_places(self, place_name, locale=None):
        """ Finds matching place API ids to use for searches.
        :param place_name: string of the place name to search for
        :kwarg locale: two letter lowercase locale string

        returns JSON response
        """
        self.path = 'places'
        self.param_str = '?term={}'.format(place_name)
        if locale:
            self.param_str += '&locale={}'.format(locale)
        return self.get_request()
    
    def search_airports(self, iata_code, locale=None):
        """ gets lon and lat of given airport
        returns JSON response
        """
        self.path = 'locations'
        self.param_str = '?term={}'.format(iata_code)
        if locale:
            self.param_str += '&locale={}'.format(locale)
        return self.get_request()

    def search_flights(self, origin, start_date, end_date):
        """ Searches for direct flights given a time range and origin airport code.
        :param origin: string representing the ID or IATA
        :param start_date: datetime representing first possible travel date
        :param end_date: datetime representing last possible travel date

        returns JSON response
        """
        self.path = 'flights'
        self.param_str = '?flyFrom={}&dateFrom={}&dateTo={}&directFlights={}'.format(
                origin, start_date.strftime('%d/%m/%Y'),
                end_date.strftime('%d/%m/%Y'),1)

        resp = self.get_request()
        flights = []
        for flight in resp.get('data'):
            flight_info = {
                'departure': datetime.utcfromtimestamp(flight.get('dTimeUTC')),
                'arrival': datetime.utcfromtimestamp(flight.get('aTimeUTC')),
                'price': flight.get('price'),
                'currency': resp.get('currency'),
                'legs': []
            }
            flight_info['duration'] = flight_info['arrival'] - \
                flight_info['departure']
            flight_info['duration_hours'] = (flight_info[
                'duration'].total_seconds() / 60.0) / 60.0
            for route in flight['route']:
                flight_info['legs'].append({
                    'carrier': route['airline'],
                    'departure': datetime.utcfromtimestamp(
                        route.get('dTimeUTC')),
                    'arrival': datetime.utcfromtimestamp(
                        route.get('aTimeUTC')),
                    'from': '{} ({})'.format(route['cityFrom'],
                                             route['flyFrom']),
                    'to': '{} ({})'.format(route['cityTo'], route['flyTo']),
                })
            flight_info['carrier'] = ', '.join(set([c.get('carrier') for c
                                                    in flight_info['legs']]))
            flights.append(flight_info)
        return flights

In [62]:
origin = 'KEF'    #origin airport IATA code
travel_date = '07/20/2018'   
sp_api = SkyPickerApi()
sp_results = sp_api.search_flights(origin, datetime.strptime(travel_date, '%m/%d/%Y'),
                                       datetime.strptime(travel_date, '%m/%d/%Y')+timedelta(days=7))



In [63]:
dest_iata_codes = []
for i in range(len(sp_results)):
    dest_iata_code = sp_results[i]['legs'][0]['to'][-4:-1]
    dest_iata_codes.append(dest_iata_code)
    
unique_dest_iata_codes = set(dest_iata_codes)
print(unique_dest_iata_codes)

{'STL', 'ARN', 'GOH', 'RIX', 'WAW', 'BCN', 'BGO', 'GLA', 'JAV', 'PRG', 'MAD', 'HEL', 'BLL', 'MCI', 'MUC', 'EDI', 'CGN', 'YHZ', 'GVA', 'EWR', 'DME', 'ORD', 'DUS', 'PIT', 'ANC', 'LAX', 'PDX', 'CPH', 'CVG', 'POZ', 'WRO', 'LYS', 'SXF', 'KTW', 'BRE', 'HAM', 'GDN', 'AMS', 'DEN', 'MAN', 'SFO', 'TLV', 'DUB', 'BUD', 'MXP', 'VIE', 'BRU', 'YVR', 'VNO', 'DFW', 'BOS', 'TFS', 'YEG', 'PMI', 'ZRH', 'GOT', 'BSL', 'FRA', 'AGP', 'OSL', 'ALC', 'BWI', 'CLE', 'DTW', 'LTN', 'DRS', 'NUE', 'YUL', 'PHL', 'SEA', 'YYZ', 'MSP', 'CDG', 'TRS'}


In [64]:
def get_airport_coords(sp_api,iata_code):
    lat = None; lon = None;  #reset values
    airport_results = sp_api.search_airports(iata_code)
    for airport_result in airport_results['locations']:
        if airport_result['id'] != iata_code:
            continue
        lat = airport_result['location']['lat']
        lon = airport_result['location']['lon']
    return lat,lon


In [65]:
sp_loc_api = SkyPickerApi()
unique_dest_lons = []
unique_dest_lats = []
        
for iata_code in unique_dest_iata_codes:
    lat,lon = get_airport_coords(sp_api,iata_code)
    unique_dest_lats.append(lat)
    unique_dest_lons.append(lon)

In [68]:
origin_lat,origin_lon = get_airport_coords(sp_api,origin)


63.985 -22.605556


In [66]:
dest_airports = pd.DataFrame(data=np.transpose([list(unique_dest_iata_codes),unique_dest_lons,unique_dest_lats]),
                                    columns=['code','lon','lat'])

In [67]:
dest_airports

,code,lon,lat
0,STL,-90.37,38.748611
1,ARN,17.918611,59.651944
2,GOH,-51.678056,64.190833
3,RIX,23.971111,56.923611
4,WAW,20.967222,52.165833
5,BCN,2.078333,41.296944
6,BGO,5.218056,60.293333
7,GLA,-4.433333,55.871944
8,JAV,-51.058056,69.242778
9,PRG,14.26,50.100833


In [6]:
### This is a dump of the code from the GeodesicLinesToGIS module, which I cannot import 
### for some reason due to circular dependencies.
### source: https://github.com/GeographicaGS/GeodesicLinesToGIS/blob/master/geodesiclinestogis/geodesicline2gisfile.py

class ComputeGeodesicLineError(Exception):
    pass

class ExportGeodesicLineError(Exception):
    pass

class GeodesicLine2Gisfile(object):
    """
    Compute geodesic line from start start lon/lat to
    end lon/lat and dumps to a gis file (Shapefile
    and GeoJSON). The problem of geodesic lines crossing
    antimeridian is solved.
    Author: Cayetano Benavent, 2015.
    https://github.com/GeographicaGS/GeodesicLinesToGIS
    Usage is very simple. There are two modes:
        - Single input (one start/end).
        - Multiple input (more than one start/end).
    Single input example:
        >>> from geodesiclinestogis import GeodesicLine2Gisfile
        >>> lons_lats = (-3.6,40.5,-118.4,33.9)
        >>> folderpath = '/tmp'
        >>> layername = "geodesicline"
        >>> gtg = GeodesicLine2Gisfile()
        >>> cd = gtg.gdlComp(lons_lats, km_pts=30)
        #shapefile output
        >>> gtg.gdlToGisFile(cd, folderpath, layername)
        #geojson output
        >>> gtg.gdlToGisFile(cd, folderpath, layername, fmt="GeoJSON")
    Multiple input example:
        >>> from geodesiclinestogis import GeodesicLine2Gisfile
        >>> data = [
                (-6.,37.,-145.,11.),
                (-150.,37.,140.,11.),
                (-6.,37.,120.,50.),
                (-3.6,40.5,-118.4,33.9),
                (-118.4,33.9,139.8,35.5),
                (-118.4,33.9,104.,1.35),
                (-118.4,33.9,151.,-33.9),
                (-20.4,33.9,178.,-33.9)
            ]
        >>> folderpath = "/tmp/geod_line"
        >>> layername = "geodesicline"
        >>> gtg = GeodesicLine2Gisfile()
        >>> gtg.gdlToGisFileMulti(data, folderpath, layername)
    """


    def __init__(self, antimeridian=True, loglevel="INFO"):
        """
            antimeridian: solving antimeridian problem [True/False].
            prints: print output messages [True/False].
        """
        self.__antimeridian = antimeridian
        self.__logger = self.__loggerInit(loglevel)
        self.__distances = []


    def __loggerInit(self, loglevel):
        """
        Logger init...
        """
        if loglevel=="INFO":
            __log_level=logging.INFO
        elif loglevel=="DEBUG":
            __log_level=logging.DEBUG
        elif loglevel=="ERROR":
            __log_level=logging.ERROR
        else:
            __log_level=logging.NOTSET

        logfmt = "[%(asctime)s - %(levelname)s] - %(message)s"
        dtfmt = "%Y-%m-%d %I:%M:%S"

        logging.basicConfig(level=__log_level, format=logfmt, datefmt=dtfmt)

        return logging.getLogger()
    
    @property
    def distances(self):
        return self.__distances
    
    def __dest_folder(self, dest, crtfld):
        if not os.path.exists(dest):
            if not crtfld:
                self.__logger.error("Output folder does not exist. Set a valid folder path to store file.")
                return
            os.mkdir(dest)
            self.__logger.debug("New output folder {0} created.".format(dest))
        else:
            self.__logger.debug("Output folder {0} already exists.".format(dest))

    def gdlComp(self, lons_lats, km_pts=20):
        """
        Compute geodesic line
            lons_lats: input coordinates.
            (start longitude, start latitude, end longitude, end latitude)
            km_pts: compute one point each 20 km (default).
        """

        try:
            lon_1, lat_1, lon_2, lat_2 = lons_lats

            pygd = Geod(ellps='WGS84')

            res = pygd.inv(lon_1, lat_1, lon_2, lat_2)
            dist = res[2]

            pts  = int(math.ceil(dist) / (km_pts * 1000))

            coords = pygd.npts(lon_1, lat_1, lon_2, lat_2, pts)

            coords_se = [(lon_1, lat_1)] + coords
            coords_se.append((lon_2, lat_2))
            
            self.__distances.append({
                "id": len(self.__distances),
                "distance": dist,
                "coords": lons_lats
            })
            
            self.__logger.info("Geodesic line succesfully created!")
            self.__logger.info("Total points = {:,}".format(pts))
            self.__logger.info("{:,.4f} km".format(dist / 1000.))

            return coords_se

        except Exception as e:
            self.__logger.error("Error: {0}".format(e))
            raise ComputeGeodesicLineError(e)


    def gdlToGisFile(self, coords, folderpath, layername, fmt="ESRI Shapefile",
                     epsg_cd=4326, prop=None, crtfld=True):
        """
        Dump geodesic line coords to ESRI Shapefile
        and GeoJSON Linestring Feature
            coords: input coords returned by gcComp.
            folderpath: folder to store output file.
            layername: output filename.
            fmt: output format ("ESRI Shapefile" (default), "GeoJSON").
            epsg_cd: Coordinate Reference System, EPSG code (default: 4326)
            prop: property
            
            crtfld: create folder if not exists (default: True).
        """

        schema = { 'geometry': 'LineString',
                   'properties': { 'prop': 'str' }
                   }

        try:

            if fmt in ["ESRI Shapefile", "GeoJSON"]:
                ext = ".shp"
                if fmt == "GeoJSON":
                    ext = ".geojson"

                filepath = os.path.join(folderpath, "{0}{1}".format(layername, ext))

                self.__dest_folder(folderpath, crtfld)

                if fmt == "GeoJSON" and os.path.isfile(filepath):
                    os.remove(filepath)

                out_crs = from_epsg(epsg_cd)

                with collection(filepath, "w", fmt, schema, crs=out_crs) as output:

                    line = LineString(coords)

                    geom = mapping(line)

                    if self.__antimeridian:
                        line_t = self.__antiMeridianCut(geom)
                    else:
                        line_t = geom

                    output.write({
                        'properties': {
                            'prop': prop
                        },
                        'geometry': line_t
                    })

                self.__logger.info("{0} succesfully created!".format(fmt))

            else:
                self.__logger.error("No format to store output...")
                return

        except Exception as e:
            self.__logger.error("Error: {0}".format(e))
            raise ExportGeodesicLineError(e)


    def gdlToGisFileMulti(self, data, folderpath, layername, prop=[], gjs=True):
        """
        Run creation for a multi input: a list of lat/lon.
            data: a list with input coordinates.
            [
              (start longitude, start latitude, end longitude, end latitude),
              (start longitude, start latitude, end longitude, end latitude),
              (start longitude, start latitude, end longitude, end latitude),
              (start longitude, start latitude, end longitude, end latitude),
              ...
            ]
            folderpath: folder to store output files.
            layername: output base filename (an ordinal integer is added at the end).
            gfs: GeoJSON output format [True (default)|False], in addition to Shapefile.
        """

        try:
            lendata = len(data)

            for i in range(lendata):
                lyrnm = "{0}{1}".format(layername, i)
                _prop = prop[i] if prop else None
                self.__multiGeodesicLineCreation(data[i], folderpath, lyrnm, gjs, _prop)

        except Exception as e:
            self.__logger.error("Error: {0}".format(e))
            raise ExportGeodesicLineError(e)


    def __multiGeodesicLineCreation(self, lons_lats, folderpath, layername, gjs, prop):
        """
        Creating geodesic lines in batch mode
        """

        cd = self.gdlComp(lons_lats)

        self.gdlToGisFile(cd, folderpath, layername, prop=prop)

        if gjs:
            self.gdlToGisFile(cd, folderpath, layername, fmt="GeoJSON")


    def __antiMeridianCut(self, geom):
        """
        Solving antimeridian problem.
        """

        src_crs = '+proj=longlat +datum=WGS84 +no_defs'
        dst_crs = '+proj=longlat +datum=WGS84 +no_defs'

        am_offset = 360.0

        line_t = transform_geom(src_crs, dst_crs, geom,
                                antimeridian_cutting=self.__antimeridian,
                                antimeridian_offset=am_offset,
                                precision=-1)

        return line_t

In [59]:
dest_airports.loc[5].code
dest_airports.loc[5].lat



'41.296944'

In [72]:
arctic_circle_lat = 64.0   #it's actually 66.3 degrees, but give a bit of wiggle room
if origin_lat > arctic_circle_lat:
    does_this_route_pass_through_arctic = np.ones(len(dest_airports))
else:
    does_this_route_pass_through_arctic = np.zeros(len(dest_airports))
    for i in range(len(dest_airports)):
        folderpath = '/Users/michevan/Desktop/insight/gis'
        layername = origin+'-'+dest_airports.loc[i].code
        gtg = GeodesicLine2Gisfile()
        lons_lats = (origin_lon, origin_lat, np.float(dest_airports.loc[i].lon), np.float(dest_airports.loc[i].lat))
        cd = gtg.gdlComp(lons_lats, km_pts=10)
        gtg.gdlToGisFile(cd, folderpath, layername, fmt="GeoJSON")  # geojson output
        gc_lat = []; gc_lon = []
        with open(folderpath+'/'+layername+'.geojson') as json_file:  
            data = json.load(json_file)
            for j in range(len(data['features'][0]['geometry']['coordinates'][:])):
                lon,lat = data['features'][0]['geometry']['coordinates'][j]
                gc_lat.append(lat)
                gc_lon.append(lon)
        if np.max(gc_lat) > arctic_circle_lat:
            does_this_route_pass_through_arctic[i] = 1





[2018-06-11 03:09:50 - INFO] - Geodesic line succesfully created!
[2018-06-11 03:09:50 - INFO] - Total points = 515
[2018-06-11 03:09:50 - INFO] - 5,150.9773 km
[2018-06-11 03:09:50 - WARNING] - CPLE_NotSupported in b'dataset /Users/michevan/Desktop/insight/gis/STL.geojson does not support layer creation option ENCODING'
[2018-06-11 03:09:50 - INFO] - GeoJSON succesfully created!
[2018-06-11 03:09:50 - INFO] - Geodesic line succesfully created!
[2018-06-11 03:09:50 - INFO] - Total points = 215
[2018-06-11 03:09:50 - INFO] - 2,150.3336 km
[2018-06-11 03:09:50 - WARNING] - CPLE_NotSupported in b'dataset /Users/michevan/Desktop/insight/gis/ARN.geojson does not support layer creation option ENCODING'
[2018-06-11 03:09:50 - INFO] - GeoJSON succesfully created!
[2018-06-11 03:09:50 - INFO] - Geodesic line succesfully created!
[2018-06-11 03:09:50 - INFO] - Total points = 140
[2018-06-11 03:09:50 - INFO] - 1,405.9376 km
[2018-06-11 03:09:50 - WARNING] - CPLE_NotSupported in b'dataset /Users/m

[2018-06-11 03:09:50 - WARNING] - CPLE_NotSupported in b'dataset /Users/michevan/Desktop/insight/gis/ORD.geojson does not support layer creation option ENCODING'
[2018-06-11 03:09:50 - INFO] - GeoJSON succesfully created!
[2018-06-11 03:09:50 - INFO] - Geodesic line succesfully created!
[2018-06-11 03:09:50 - INFO] - Total points = 221
[2018-06-11 03:09:50 - INFO] - 2,218.7231 km
[2018-06-11 03:09:50 - WARNING] - CPLE_NotSupported in b'dataset /Users/michevan/Desktop/insight/gis/DUS.geojson does not support layer creation option ENCODING'
[2018-06-11 03:09:50 - INFO] - GeoJSON succesfully created!
[2018-06-11 03:09:50 - INFO] - Geodesic line succesfully created!
[2018-06-11 03:09:50 - INFO] - Total points = 449
[2018-06-11 03:09:50 - INFO] - 4,496.6841 km
[2018-06-11 03:09:50 - WARNING] - CPLE_NotSupported in b'dataset /Users/michevan/Desktop/insight/gis/PIT.geojson does not support layer creation option ENCODING'
[2018-06-11 03:09:50 - INFO] - GeoJSON succesfully created!
[2018-06-11 

[2018-06-11 03:09:50 - INFO] - GeoJSON succesfully created!
[2018-06-11 03:09:50 - INFO] - Geodesic line succesfully created!
[2018-06-11 03:09:50 - INFO] - Total points = 312
[2018-06-11 03:09:50 - INFO] - 3,122.8527 km
[2018-06-11 03:09:50 - WARNING] - CPLE_NotSupported in b'dataset /Users/michevan/Desktop/insight/gis/BUD.geojson does not support layer creation option ENCODING'
[2018-06-11 03:09:50 - INFO] - GeoJSON succesfully created!
[2018-06-11 03:09:50 - INFO] - Geodesic line succesfully created!
[2018-06-11 03:09:50 - INFO] - Total points = 281
[2018-06-11 03:09:50 - INFO] - 2,814.4486 km
[2018-06-11 03:09:50 - WARNING] - CPLE_NotSupported in b'dataset /Users/michevan/Desktop/insight/gis/MXP.geojson does not support layer creation option ENCODING'
[2018-06-11 03:09:50 - INFO] - GeoJSON succesfully created!
[2018-06-11 03:09:50 - INFO] - Geodesic line succesfully created!
[2018-06-11 03:09:50 - INFO] - Total points = 293
[2018-06-11 03:09:50 - INFO] - 2,939.0607 km
[2018-06-11 0

[2018-06-11 03:09:51 - INFO] - 1,866.4474 km
[2018-06-11 03:09:51 - WARNING] - CPLE_NotSupported in b'dataset /Users/michevan/Desktop/insight/gis/LTN.geojson does not support layer creation option ENCODING'
[2018-06-11 03:09:51 - INFO] - GeoJSON succesfully created!
[2018-06-11 03:09:51 - INFO] - Geodesic line succesfully created!
[2018-06-11 03:09:51 - INFO] - Total points = 255
[2018-06-11 03:09:51 - INFO] - 2,551.3057 km
[2018-06-11 03:09:51 - WARNING] - CPLE_NotSupported in b'dataset /Users/michevan/Desktop/insight/gis/DRS.geojson does not support layer creation option ENCODING'
[2018-06-11 03:09:51 - INFO] - GeoJSON succesfully created!
[2018-06-11 03:09:51 - INFO] - Geodesic line succesfully created!
[2018-06-11 03:09:51 - INFO] - Total points = 256
[2018-06-11 03:09:51 - INFO] - 2,565.7298 km
[2018-06-11 03:09:51 - WARNING] - CPLE_NotSupported in b'dataset /Users/michevan/Desktop/insight/gis/NUE.geojson does not support layer creation option ENCODING'
[2018-06-11 03:09:51 - INFO

In [73]:
does_this_route_pass_through_arctic


array([ 0.,  1.,  1.,  1.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  1.,  0.,
        1.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  1.,  1.,
        1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,
        0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  1.,  0.,  0.,
        1.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  1.,  0.,  1.,  0.,  0.])

54.912216370679353